# 有価証券報告書の取得・分析処理

In [ ]:
!git clone https://github.com/kMUTOU/edinet_parser.git
%cd ./edinet_parser/
!pip install ipywidgets requests aiohttp pandas numpy xmltodict lxml

In [ ]:
import os
import re
import io
import glob
import sys
import copy
import zipfile
import datetime as dt
from typing import List

import pandas as pd
import numpy as np
import pathlib as plib
import requests as req
import xmltodict

from lxml import etree
import ipywidgets as widgets
from edinet_api import Edinet

from IPython.display import display as disp
from IPython.display import display_markdown

dispmd = lambda txt: display_markdown(txt, raw=True)

df_extract = pd.read_excel('./有価証券報告書XBRL抽出.xlsx')



def daterange(start_date: dt.date, end_date: dt.date) -> List[dt.date]:
    '''
    指定された開始日から終了日までの日付listを生成する関数
    '''
    dates: List[dt.date] = []
    current_date = start_date
    
    # ループは終了日（end_date）を超えるまで継続
    while current_date <= end_date:
        dates.append(current_date)
        # 次の日付へ移動
        current_date += dt.timedelta(days=1)

    return dates

## 1. Edinet APIの設定

### 1.1 Subscription Keyの入力

次のいずれかによって、Subscription Keyを設定する。
1. 環境変数: `EDINET_API_KEY`を設定している
2. `{"Subscription-Key": "<KEY>"}`と記載されたJSONファイルのパスを引数`key_path:str`に渡す。
3. 1.及び2.でSubscriptionキーがない場合は、widgetsで入力を求める。

In [ ]:
today = dt.date.today().isoformat()

SOURCE_DIR = '.'

edinet = Edinet(key_path='../edinet_api.json')

## 2. 書類一覧DBの作成

EDINET APIは、二つのREST方式APIリクエストを提供しています。
1. 書類一覧 API
2. 書類取得 API

1.の書類一覧 APIは、次の仕様により提供されており、日付別の書類一覧が取得できる。
```Plain
https://api.edinet-fsa.go.jp/api/[version]/documents.json?date=[ファイル日付]&type=[取得情報]&Subscription-Key=[API キー]
```
逆に言うと、日付別の書類一覧のみが提供されているため、提出者別の書類一覧などの情報をAPIから取得することはできない。このため、日付別の書類一覧を利用して、DBを構築して、抽出・選択の各種操作が可能となる様にする必要がある。

ここでは、過去90日の書類一覧をそれぞれ取得し、pandasのDataFrameにした後に、featherフォーマットで保存する。featherフォーマットで保存されたファイルがあれば、ファイルの取得はスキップする。

In [ ]:
if not os.path.isfile('documents_info.feather'):
    # 開始日と終了日を定義
    end_date = dt.date.today()
    start_date = end_date  + dt.timedelta(days=-90)
    date_list = [d.strftime('%Y-%m-%d') for d in daterange(start_date, end_date)]

    # 非同期関数を使用して、効率的に書類一覧情報を取得する。
    await edinet.async_get_docs(date_list)


    # featherもしくはSQLite3でデータを保持する。
    df_doc = pd.DataFrame()

    BIT_COLUMNS = [
        'withdrawalStatus',
        'docInfoEditStatus',
        'disclosureStatus',
        'xbrlFlag',
        'pdf_docFlag',
        'attachDocFlag',
        'englishDocFlag',
        'csvFlag',
        'legalStatus'
    ]


    paths = glob.glob(f'{edinet.TSV_PATH}/document_list_*.tsv.gz')
    paths.sort()

    for path in paths:
        try:
            df_tmp = pd.read_csv(path, sep='\t', dtype=str)
        except pd.errors.EmptyDataError:
            print(f'{path}は空の書類一覧情報です', file=sys.stderr)
        else:
            print(f'{path}を読み込みました({len(df_tmp)}レコード)', file=sys.stdout)
            df_doc = pd.concat([df_doc, df_tmp])


    for col in ['periodStart', 'periodEnd']:
        df_doc[col] = pd.to_datetime(df_doc[col], errors='coerce')
        df_doc[col] = df_doc[col].dt.date

    for col in ['opeDateTime', 'submitDateTime']:
        df_doc[col] = pd.to_datetime(df_doc[col], errors='coerce')

    for col in [col for col in df_doc.columns if 'Flag' in col]:
        df_doc[col] = df_doc[col].astype(pd.Int8Dtype())

    df_doc = df_doc.replace({np.nan: None, pd.NaT: None})
    df_doc = df_doc.where(pd.notnull(df_doc), None)


    cols = df_doc.select_dtypes('int8').columns
    for col in cols:
        df_doc[col] = df_doc[col].astype(str)

    # sequenceNumberは不要
    df_doc.drop(columns=['seqNumber'], inplace=True)

    df_doc.to_feather('documents_info.feather')
    disp(df_doc.head(10))

else:
    df_doc = pd.read_feather('documents_info.feather')
    disp(df_doc.head(10))


## 3. 有価証券報告書XBRLデータ(zip)をAPIで取得し、ファイルを保存

### 3.1 ipywdigetsで有価証券報告書提出者を選択

In [ ]:
filer_list = df_doc.loc[df_doc['docTypeCode'] == '120', 'filerName'].drop_duplicates().to_list()

filer_widget = widgets.Combobox(
    placeholder="提出者名",
    options=filer_list,
    description="提出者名:",
    ensure_option=True,
    continuous_update=False
)

def show_selected_filer(selected_filer):
    dispmd(f'### 選択した提出者: {selected_filer}')


ui = widgets.VBox([filer_widget])
out = widgets.interactive_output(show_selected_filer,
    {'selected_filer': filer_widget} 
)

disp(ui, out)

### 3.2 取得すべき書類一覧を作成

In [ ]:
# 読み込む有価証券報告書の条件をqueryに記述する。(docTypeCode: "120"が有価証券報告書)
df_parse_target = df_doc.query(f'filerName == "{filer_widget.value}" & docTypeCode == "120"')
disp(df_parse_target)

### 3.3 書類のダウンロード

In [ ]:
def read_file(path):
    with open(path, 'rb') as fr:
        data = fr.read()
    return data

df_parse_target = df_parse_target[[
    'docID', 
    'edinetCode', 'secCode', 'JCN', 'filerName', 'periodStart',
    'periodEnd', 'submitDateTime'
]]

df_parse_target = df_parse_target.set_index('docID')
df_parse_target['submitDateTime'] = pd.to_datetime(df_parse_target['submitDateTime'])
df_parse_target['saveDir'] = df_parse_target['submitDateTime'].dt.strftime(f'{edinet.DOC_PATH}/%Y/%m')

disp(df_parse_target)

await edinet.get_documents(df_parse_target['saveDir'].to_dict())


df_parse_target['xbrlPath'] = df_parse_target['saveDir'] + '/' + df_parse_target.index + '.zip'
df_parse_target['XBRLFILE'] = df_parse_target['xbrlPath'].apply(read_file)

## 4. ダウンロードした書類(有価証券報告書)の読込・解析

In [ ]:
class ParseXBRL(object):

    def __init__(self, zip_data: io.BytesIO=None):
        '''
        '''
        self.zip_data = zip_data
        self.target_ns_dict = {}
        self.content = None

    @staticmethod
    def __check(path):
        '''
        '''
        if path is None:
            return False

        if not os.path.isfile(path):
            return False
        else:
            return True


    def load_xbrl_from_zip(self, zip_data: io.BytesIO=None) -> bool:
        '''
        '''
        self.zip_data = zip_data or self.zip_data 

        self.content = {}

        try:
            # ZIPファイルを開く
            with zipfile.ZipFile(self.zip_data) as zip_ref:
                # 読み込みたいファイル名
                # 正規表現により、次の二つのみを書き出す。
                # XBRL/PublicDoc/jpcrp030000-asr-[ddd]_[EDINETコード]-[ddd]_[決算日;%Y-%m-%d]_[提出日;%Y-%m-%d].xbrl
                # XBRL/PublicDoc/jpcrp030000-asr-001_E23492-000_2024-09-30_01_2024-12-11_lab.xml'
                files_list = [fn for fn in zip_ref.filelist if re.match(r'.*XBRL/PublicDoc/.*(\.xbrl|_lab\.xml)$', fn.filename)]

                for fn in files_list:
                    # ZIP内の指定ファイルを開いて読み込む
                    ext = 'XBRL' if '.xbrl' in fn.filename else 'LABEL-XML'
                    with zip_ref.open(fn) as file:
                        content = file.read()  # ファイル内容をバイナリとして読み込む
                        # text_content = content.decode('utf-8')  # 必要に応じてデコード（テキストファイルの場合）
                        self.content.update({
                            ext: content
                        })
            return True
        except zipfile.BadZipFile:
            print('Zip File may be collapsed.', file=sys.stderr)
            return False


    def parse_xbrl(self) -> tuple[pd.DataFrame, dict]:

        # XBRLファイルを読み込み
        if self.content is None:
            with open(self.xbrl_path, 'rb') as file:
                xbrl_data = etree.parse(file)
        elif self.content.get('XBRL'):
            xbrl_data = etree.parse(io.BytesIO(self.content.get('XBRL')))

        # 勘定科目に基づく情報を抽出
        data = []
        root = xbrl_data.getroot()
        target_ns_dict = {ns: url for ns, url in root.nsmap.items() if 'jp' in ns}
        self.target_ns_dict = target_ns_dict

        # 勘定科目 (例: AccountTitle) に基づき要素を取得
        target_paths = ' | '.join([f'//{key}:*' for key in target_ns_dict.keys()])
        ns_from_url = {url: ns for ns, url in target_ns_dict.items()}

        for element in xbrl_data.xpath(target_paths, namespaces=root.nsmap):
            # 科目
            name_space = re.sub(r'{(.*)}.+', r'\1', element.tag)
            tag = element.tag.replace('{' + name_space + '}',  ns_from_url[name_space] + ":")
            # text
            tag_value = element.text  # 残高

            # element.attribで全ての属性が取得できる。
            context_ref = element.get('contextRef')  # 対応するcontextRef
            unit_ref = element.get('unitRef')  # 対応するunitRef
            decimals = element.get('decimals')  # 対応するdecimals

            # unit refが次のもので、decimalsが0以上の場合は、整数として処理する
            if unit_ref in ['pure', 'JPY', 'JPYPerShares', 'shares']:
                try:
                    if '.' in tag_value:
                        tag_value = float(tag_value)
                    else:
                        tag_value = int(tag_value)
                except TypeError:
                    tag_value = None
            elif unit_ref is not None and unit_ref in ['pure', 'JPY', 'JPYPerShares', 'shares']:
                raise ValueError(f'unknown unit_ref value = = {unit_ref}')

            row = {
                'tag': tag,
                'value': tag_value,
                'contextRef': context_ref,
                'unitRef': unit_ref,
                # 'decimals': decimals
            }
            if decimals:
                row.update({'decimals': int(decimals)})

            data.append(row)

        return data, target_ns_dict


    def get_label_ns(self) -> dict:

        data_labels = {}

        # elif self.content.get('XBRL'):
        #     xbrl_data = etree.parse(io.BytesIO(self.content.get('LABEL-XML')))

        for ns, url in self.target_ns_dict.items():
            if 'cor' not in ns:
                if not self.content['LABEL-XML']:
                    path = f"{self.xbrl_path.replace('.xbrl', '')}_lab.xml"
                    with open(path, 'r') as fr:
                        dict_data = xmltodict.parse(fr.read())
                else:
                    dict_data = xmltodict.parse(io.BytesIO(self.content['LABEL-XML']))

                data_labels.update({
                    # f'{ns}:{label["@id"].replace("label_", "")}': f'{label["#text"]}'
                    # for label in dict_data['link:linkbase']['link:labelLink']['link:label']
                    f'{ns}:{label["@xlink:label"].replace(ns.replace('asr-001', 'asr') + '_', "").replace("_label", "").replace("label_", "")}': f'{label["#text"]}'
                    for label in dict_data['link:linkbase']['link:labelLink']['link:label']
                })

            else:

                target_dir = os.sep.join(url.split('/')[3:])
                space, target_date = url.split('/')[-3:-1]

                # taxonomy path以下に対象となるディレクトリがなければ作成する
                if not os.path.isdir(target_dir):
                    #
                    plib.Path(target_dir).mkdir(exist_ok=True, parents=True)

                    url = '/'.join(url.split('/')[:-1])
                    url = f"{url}/label/{space}_{target_date}_lab.xml"

                    # labelファイルを取得しtaxonomy pathに保存する。
                    res = req.get(url)
                    if res.status_code == 200:
                        with open(f'{target_dir}/{space}_{target_date}_lab.xml', 'wb') as f:
                            f.write(res.content)
                    else:
                        print(f'GET: {url}, STATUS CODE = {res.status_code}', file=sys.stderr)


                if os.path.isfile(f'{target_dir}/{space}_{target_date}_lab.xml'):
                    with open(f'{target_dir}/{space}_{target_date}_lab.xml', 'r') as f:
                        # print(f'{target_dir}/{space}_{target_date}_lab.xml')
                        dict_data = xmltodict.parse(f.read())
                        data_labels.update({
                            f'{ns}:{label["@id"].replace("label_", "")}': f'{label["#text"]}'
                            for label in dict_data['link:linkbase']['link:labelLink']['link:label']
                        })

        return data_labels


def parse(data: io.BytesIO):

    # if not os.path.isfile():
    #     return None

    xbrl = ParseXBRL(data)

    # zipから必要となるXBRLファイルと_lab.xmlファイルを取得する。
    ret = xbrl.load_xbrl_from_zip()

    if not ret:
        return None
    elif xbrl.content == {}:
        print('NO Valid XBRL Contents exist', file=sys.stderr)
        return None

    # XBRLファイルをパースする。
    try:
        data, ns = xbrl.parse_xbrl()
    except:
        print(f'Unknown Error', file=sys.stderr)
        return None

    # tagから日本語ラベル名を取得する
    # print(f'target doc: {path}')
    data_labels = xbrl.get_label_ns()

    # pandas DataFrameに変換
    df = pd.DataFrame(data)

    df['label'] = df['tag'].replace(data_labels)

    ## 大株主の情報を取得する。
    df_major_sh = df[df['contextRef'].str.contains(r"CurrentYearInstant_No\d{1,2}MajorShareholdersMember", regex=True)][['contextRef', 'tag', 'value', 'label']]
    if len(df_major_sh) > 0:
        # ピボットして contextRef を行、tag を列に
        df_pivot = df_major_sh.pivot(
            index='contextRef',
            columns='label',
            values='value'
        ).reset_index()

        df_major_sh = df_pivot#.sort_values(
        df_major_sh['contextRef'] = df_major_sh['contextRef'].str.replace(r'.*No(\d{1,2}).*', r'\1', regex=True).astype(np.int8)
        df_major_sh = df_major_sh.sort_values(['contextRef'])

    else:
        df_major_sh = None


    ## 子会社(Affiliated Entities)の情報
    df_aes = pd.DataFrame()
    target_tag = 'jpcrp_cor:OverviewOfAffiliatedEntitiesTextBlock'
    try:

        affiliated_entities = df.query(f'tag == "{target_tag}"')['value'].values[0]
        df_aes = pd.read_html(io.StringIO(affiliated_entities), header=[0, 1])
        df_aes = pd.concat(df_aes, axis='index')
    except:
        print(f'Not Found OverviewOfAffiliatedEntitiesTextBlock', file=sys.stderr)


    # 連結・単体で必要となる項目が異なる。
    # 連結決算ありは"true"、連結決算なしは"false"。
    # 有価証券届出書、有価証券報告書、四半期報告書又は半期報告書以外の提出書類の場合及び経理の状況が記載されない場合は、nil。
    try:
        # consolidated => true/false
        consolidated = df.query('tag == "jpdei_cor:WhetherConsolidatedFinancialStatementsArePreparedDEI"')['value'].values[0]
        # 
        gaap = df.query('tag == "jpdei_cor:AccountingStandardsDEI"')['value'].values[0]
        dispmd(f'### 会計基準: {gaap}({"連結" if consolidated == "true" else "単体"})')

        df['unitRef'] = df['unitRef'].str.upper()
        df.loc[df['unitRef'].isin(['JPY', 'PURE', 'JPYPERSHARES', 'SHARES']), 'value'] = pd.to_numeric(df.loc[df['unitRef'].isin(['JPY', 'PURE', 'JPYPERSHARES', 'SHARES']), 'value'])

    except:
        print(f'Not Found WhetherConsolidatedFinancialStatementsArePreparedDEI tag Error', file=sys.stderr)

    if gaap == 'Japan GAAP':
        target_tags = [
            'jpcrp_cor:NetSalesSummaryOfBusinessResults', # 売上高、経営指標等
            'jpcrp_cor:ProfitLossAttributableToOwnersOfParentSummaryOfBusinessResults', # 親会社株主に帰属する当期純利益又は親会社株主に帰属する当期純損失（△）、経営指標等
            'jpcrp_cor:ComprehensiveIncomeSummaryOfBusinessResults', # 包括利益、経営指標等
            'jpcrp_cor:RateOfReturnOnEquitySummaryOfBusinessResults', # 自己資本利益率、経営指標等
            'jpcrp_cor:NetCashProvidedByUsedInOperatingActivitiesSummaryOfBusinessResults', # 営業活動によるキャッシュ・フロー、経営指標等
            'jpcrp_cor:NetCashProvidedByUsedInInvestingActivitiesSummaryOfBusinessResults',	# 投資活動によるキャッシュ・フロー、経営指標等
            'jpcrp_cor:NetCashProvidedByUsedInFinancingActivitiesSummaryOfBusinessResults', # 財務活動によるキャッシュ・フロー、経営指標等
            'jpcrp_cor:CashAndCashEquivalentsSummaryOfBusinessResults' # 現金及び現金同等物、経営指標等
        ]
    elif gaap == 'IFRS':
        target_tags = [
            # IFRS
            'jpcrp_cor:RevenueIFRSSummaryOfBusinessResults', # 売上収益
            'jpcrp_cor:ProfitLossAttributableToOwnersOfParentIFRSSummaryOfBusinessResults', # 当期利益又は当期損失（△）：親会社の所有者に帰属
            'jpcrp_cor:EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults', # 親会社の所有者に帰属する持分
            'jpcrp_cor:TotalAssetsIFRSSummaryOfBusinessResults', # 総資産額
            'jpcrp_cor:RateOfReturnOnEquityIFRSSummaryOfBusinessResults', # 自己資本利益率、経営指標等
            'jpcrp_cor:RatioOfOwnersEquityToGrossAssetsIFRSSummaryOfBusinessResults', # IFRSの場合の自己資本比率

            # 株価収益率は、連結・単体にも記載があったがどちらを利用すればいいかわからない。
            # jpcrp_cor:PriceEarningsRatioIFRSSummaryOfBusinessResults, 株価収益率
            'jpcrp_cor:CashFlowsFromUsedInOperatingActivitiesIFRSSummaryOfBusinessResults', # 営業活動によるキャッシュ・フロー
            'jpcrp_cor:CashFlowsFromUsedInInvestingActivitiesIFRSSummaryOfBusinessResults', # 投資活動によるキャッシュ・フロー
            'jpcrp_cor:CashFlowsFromUsedInFinancingActivitiesIFRSSummaryOfBusinessResults', # 財務活動によるキャッシュ・フロー
            'jpcrp_cor:CashAndCashEquivalentsIFRSSummaryOfBusinessResults' # 現金及び現金同等物
        ]
    elif gaap == 'US GAAP':
        target_tags = [
            'jpcrp_cor:RevenuesUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:ProfitLossBeforeTaxUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:NetIncomeLossAttributableToOwnersOfParentUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:ComprehensiveIncomeAttributableToOwnersOfParentUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:EquityAttributableToOwnersOfParentUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:EquityIncludingPortionAttributableToNonControllingInterestUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:PriceEarningsRatioUSGAAPSummaryOfBusinessResults',
            
            'jpcrp_cor:CashFlowsFromUsedInOperatingActivitiesUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:CashFlowsFromUsedInInvestingActivitiesUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:CashFlowsFromUsedInFinancingActivitiesUSGAAPSummaryOfBusinessResults',
            'jpcrp_cor:CashAndCashEquivalentsUSGAAPSummaryOfBusinessResults',
        ]
    df_div = df.loc[df['tag'].isin(target_tags), 
                    [#'tag', 
                     'contextRef', 'label', 'value', 'unitRef', 'decimals']]
    
    df_div = df_div.query('~contextRef.str.contains("NonConsolidatedMember")')

    target_tags = [
        ## 配当額
        # 提出会社に記載される内容なので、NonConsolidatedMember
        'jpcrp_cor:DividendPaidPerShareSummaryOfBusinessResults', # １株当たり配当額、経営指標等
        'jpcrp_cor:InterimDividendPaidPerShareSummaryOfBusinessResults', # １株当たり中間配当額、経営指標等
        'jpcrp_cor:PayoutRatioSummaryOfBusinessResults', # 配当性向、経営指標等
        'jpcrp_cor:TotalShareholderReturn', # 
        'jpcrp_cor:TotalNumberOfIssuedSharesSummaryOfBusinessResults' # 発行済株式
    ]

    df_append = df.loc[df['tag'].isin(target_tags), 
                       ['contextRef', 'label', 'value', 'unitRef', 'decimals']]
    
    df_div = pd.concat([df_div, df_append], axis='index', ignore_index=True)

    # df[df[['A', 'B']].apply(tuple, axis=1).isin(targets)]

    df_div['contextRef'] = df_div['contextRef'].str.replace(r'(.*Year).*', r'\1', regex=True)

    df_div = df_div.pivot(
        index='label',
        columns='contextRef',
        values='value'
    ).reset_index()

    df_add_info = {}
    for tgt in ['CurrentYear', 'Prior1Year']:
        df_extract_target = copy.copy(df_extract)
        df_extract_target['contextRef'] = df_extract_target['contextRef'].str.replace('CurrentYear', tgt)
        df_add_info[tgt] = df.merge(df_extract_target, how='inner', on=['contextRef', 'tag']).drop_duplicates()
        df_add_info[tgt] = df_add_info[tgt].set_index(['output_label'])['value'].to_dict()
    
    return {
        'major_shareholders': df_major_sh, 
        'frame': {
            '指標': df_div,
            '当期': df_add_info['CurrentYear'],
            '前期': df_add_info['Prior1Year'], 
        },
        'affiliated_entities': df_aes
    }

In [ ]:
ret = df_parse_target['XBRLFILE'].apply(lambda data: parse(io.BytesIO(data)))

disp(ret[0]['major_shareholders'])
disp(ret[0]['frame']['指標'])

df_info = {}
for tgt in ['当期', '前期']:
    stock_info = {}
    for row_num in range(len(df_parse_target)):
        stock_info.update({
            df_parse_target['periodEnd'][row_num].isoformat(): ret[row_num]['frame'][tgt]
        })

    df_tmp = pd.DataFrame(stock_info)
    df_tmp = df_tmp.transpose()
    df_tmp = df_tmp.fillna(0)
    df_tmp.index = pd.to_datetime(df_tmp.index)
    disp(df_tmp)
    
    df_info[tgt] = df_tmp

In [ ]:
df_info = {}
for tgt in ['当期', '前期']:
    stock_info = {}
    for row_num in range(len(df_parse_target)):
        stock_info.update({
            df_parse_target['periodEnd'][row_num].isoformat(): ret[row_num]['frame'][tgt]
        })

    df_tmp = pd.DataFrame(stock_info)
    df_tmp = df_tmp.transpose()
    df_tmp = df_tmp.fillna(0)
    df_tmp.index = pd.to_datetime(df_tmp.index)
    # disp(df_tmp)
    df_info[tgt] = df_tmp

df_info = df_info['当期'].join(df_info['前期'], lsuffix='_当期', rsuffix='_前期')
disp(df_info)


In [ ]:
for term in ['当期', '前期']:
    df_info[f'有利子負債_{term}'] = 0
    for col in [f'短期借入金', '社債・借入金(流動)', '社債・借入金(固定)']:
        if col + f'_{term}' in df_info.columns:
            df_info[f'有利子負債_{term}'] += df_info[f'{col}_{term}']

    df_info[f'投下資本_{term}'] = df_info[f'有利子負債_{term}'] + df_info[f'株主資本_{term}']
    for col in [f'剰余金の配当']:
        if col + f'_{term}' in df_info.columns:
            df_info['配当性向'] = -df_info[f'{col}_{term}'] / df_info[f'親会社株主に帰属する当期純利益_{term}']

df_info['RoE'] = df_info['親会社株主に帰属する当期純利益_当期'] / (df_info['株主資本_前期'] + df_info['株主資本_当期']) / 2 
if '剰余金の配当_当期' in df_info.columns:
    df_info['DoE'] = -df_info['剰余金の配当_当期']  / (df_info['株主資本_前期'] + df_info['株主資本_当期']) / 2 

# Returnには、当期純利益(包括利益)を利用
df_info['RoA'] = df_info['当期純利益_当期']  / (df_info['株主資本_前期'] + df_info['株主資本_当期']) / 2 

df_info['RoIC'] = df_info['親会社株主に帰属する当期純利益_当期'] / (df_info['投下資本_前期'] + df_info['投下資本_当期']) / 2 

disp(df_info[[#'従業員数(連結)', '従業員数(単体)', 
             '資本金_当期', '資本剰余金_当期', '利益剰余金_当期', '株主資本_当期', 
             '有利子負債_当期', 
             '投下資本_当期', 
             '親会社株主に帰属する当期純利益_当期', 
             'RoA', 'RoE', 'RoIC',              
            #  '剰余金の配当_当期',
            #  '配当性向', 'DoE'
             ]])

## 5. RoE / RoIC推移の可視化


In [ ]:
!pip plotly

In [ ]:
import plotly.express as px

# 折れ線グラフを作成
fig = px.line(df_info[['RoA', 'RoE', 'RoIC']], markers=True)
fig.update_traces(
    marker=dict(size=8, symbol="circle"),
    line=dict(width=2),
    textposition="top center"  # テキストの位置（上中央など）
)

# グラフを表示
dispmd('### 収益指標(RoA / RoE / RoIC)')
fig.show()

# 折れ線グラフを作成
fig = px.line(df_info[['RoA', 'RoE', 'RoIC']], markers=True)
fig.update_traces(
    marker=dict(size=8, symbol="circle"),
    line=dict(width=2),
    textposition="top center"  # テキストの位置（上中央など）
)

# グラフを表示
dispmd('### 収益指標(RoA / RoE / RoIC)')
fig.show()